## **Modules**

In [ ]:
# Import classes from source files
from utils1_extract import DataExtractor
from utils2_transform import DataProcessor
from utils3_load import DatabaseConnector

In [ ]:
# Create an instance for each class
extractor = DataExtractor()
processor = DataProcessor()
connector = DatabaseConnector()

## **Settings**

In [ ]:
# Read database credentials as dictionary
aws_creds = connector.read_creds("aws_creds.yaml")
local_creds = connector.read_creds("local_creds.yaml")

In [ ]:
# Connect to the database engine using the credentials
engine = connector.init_engine(aws_creds)

In [ ]:
# Retrieve the name of the tables in the database
tables_list = connector.list_tables(engine)

## **Processing**

### **Users**

In [ ]:
# Extract and read user data from the database
user_details = extractor.read_table(engine, "legacy_users")

In [ ]:
# Perform the cleaning of user data
user_data = processor.clean_users(user_details)

In [ ]:
# Use the method to upload user data in the database
connector.upload_db(local_creds, user_data, "users_table")

### **Payments**

In [ ]:
# Extract and read card details from a PDF document
card_details = extractor.retrieve_pdf(
    link = "https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf")

In [ ]:
# Perform the cleaning of card data
card_data = processor.clean_cards(card_details)

In [ ]:
# Use the method to upload card details in the database
connector.upload_db(local_creds, card_data, "cards_table")

### **Stores**

In [ ]:
# Extract the number of stores and information from the API
url_1 = "https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores"
url_2 = "https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details"
headers = {"x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"}

number_of_stores = extractor.list_stores(url_1, headers)
store_details = extractor.retrieve_stores(number_of_stores, url_2, headers)

In [ ]:
# Perform the cleaning of store data
store_data = processor.clean_stores(store_details)

In [ ]:
# Use the method to upload store details in the database
connector.upload_db(local_creds, store_data, "stores_table")

### **Products**

In [ ]:
# Extract product information stored in an S3 bucket on AWS
product_details = extractor.extract_s3(
    bucket_name = "data-handling-public",
    object_name = "products.csv",
    file_name = "products.csv")

In [ ]:
# Convert product weight to decimal value
product_details = processor.convert_weights(product_details)

In [ ]:
# Perform the cleaning of product data
product_data = processor.clean_products(product_details)

In [ ]:
# Use the method to upload store details in the database
connector.upload_db(local_creds, product_data, "products_table")

### **Orders**

In [ ]:
# Extract and read orders data from the database
orders_details = extractor.read_table(engine, "orders_table")

In [ ]:
# Perform the cleaning of orders data
orders_data = processor.clean_orders(orders_details)

In [ ]:
# Use the method to upload orders data in the database
connector.upload_db(local_creds, orders_data, "orders_table")

### **Events**

In [ ]:
# Extract events data stored in an S3 bucket on AWS
date_events = extractor.extract_events(
    bucket_name = "data-handling-public",
    object_name = "date_details.json",
    file_name = "date_details.json")

In [ ]:
# Perform the cleaning of product data
events_data = processor.clean_events(date_events)

In [ ]:
# Use the method to upload store details in the database
connector.upload_db(local_creds, events_data, "events_table")